In [ ]:
import torch
from transformer_lens import HookedTransformer
import numpy as np 
from torch import nn
import pickle

# autoencoder_filename = "/home/mchorse/logan/sparse_coding/output_sweep_tied_mlpout_l1_r8/_9/learned_dicts.pt"
autoencoder_filename = "/home/mchorse/sparse_coding_hoagy/tiedlong_tied_residual_l4_r4/_80/learned_dicts.pt"
auto_num = 5 # Selects which specific autoencoder to use
all_autoencoders = torch.load(autoencoder_filename)
num_dictionaries = len(all_autoencoders)
print(f"Loaded {num_dictionaries} autoencoders")
smaller_autoencoder, hyperparams = all_autoencoders[auto_num]
l1_alpha = hyperparams['l1_alpha']
larger_autoencoder, hyperparams2 = all_autoencoders[auto_num+1]
smaller_dict = smaller_autoencoder.get_learned_dict()
larger_dict = larger_autoencoder.get_learned_dict()
print(f"Hyperparams: {hyperparams}, {hyperparams2}")
#Change these settings to load the correct autoencoder
layer = 4
setting = "residual"
# setting = "attention"
# setting = "mlp"
# setting = "mlp_out"
model_name = "EleutherAI/pythia-70m-deduped"
# model_name = "EleutherAI/pythia-160m"

device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
model = HookedTransformer.from_pretrained(model_name, device=device)

if setting == "residual":
    cache_name = f"blocks.{layer}.hook_resid_post"
    neurons = model.cfg.d_model
elif setting == "mlp":
    cache_name = f"blocks.{layer}.mlp.hook_post"
    neurons = model.cfg.d_mlp
elif setting == "attention":
    cache_name = f"blocks.{layer}.hook_attn_out"
    neurons = model.cfg.d_model
elif setting == "mlp_out":
    cache_name = f"blocks.{layer}.hook_mlp_out"
    neurons = model.cfg.d_model
else:
    raise NotImplementedError

In [ ]:
# filename_pca = "/mnt/ssd-cluster/baselines/l4_residual/pca_topk.pt"
# pca_topk = torch.load(filename_pca)
# filename_ica = "/mnt/ssd-cluster/baselines/l4_residual/ica_topk.pt"
# ica_topk = torch.load(filename_ica)
# pca_dict = pca_topk.get_learned_dict()
# ica_dict = ica_topk.get_learned_dict()

In [ ]:
from scipy.optimize import linear_sum_assignment
import matplotlib.pyplot as plt
#Dictionary Comparison
smaller_dict_features, _ = smaller_dict.shape
larger_dict_features, _ = larger_dict.shape
larger_dict = larger_dict.to(device)
# Hungary algorithm
# Calculate all cosine similarities and store in a 2D array
cos_sims = np.zeros((smaller_dict_features, larger_dict_features))
for idx, vector in enumerate(smaller_dict):
    cos_sims[idx] = torch.nn.functional.cosine_similarity(vector.to(device), larger_dict, dim=1).cpu().numpy()
# Convert to a minimization problem
cos_sims = 1 - cos_sims
# Use the Hungarian algorithm to solve the assignment problem
row_ind, col_ind = linear_sum_assignment(cos_sims)
# Retrieve the max cosine similarities and corresponding indices
max_cosine_similarities = 1 - cos_sims[row_ind, col_ind]

# Get the indices of the max cosine similarities in descending order
max_indices = np.argsort(max_cosine_similarities)[::-1].copy()
print(("# of features above 0.9:", (max_cosine_similarities > .9).sum()))
# Plot histogram of max_cosine_similarities
plt.hist(max_cosine_similarities, bins=20)
plt.show()

In [ ]:
# Downnload dataset
from datasets import Dataset, load_dataset
dataset_name = "NeelNanda/pile-10k"
token_amount= 40
dataset = load_dataset(dataset_name, split="train").map(
    lambda x: model.tokenizer(x['text']),
    batched=True,
).filter(
    lambda x: len(x['input_ids']) > token_amount
).map(
    lambda x: {'input_ids': x['input_ids'][:token_amount]}
)

In [ ]:
# Now we can use the model to get the activations
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from einops import rearrange
# neurons = model.W_in.shape[-1]
neurons = model.cfg.d_model
datapoints = dataset.num_rows
batch_size = 64
neuron_activations = torch.zeros((datapoints*token_amount, neurons))
dictionary_activations = torch.zeros((datapoints*token_amount, smaller_dict_features))
smaller_autoencoder.to_device(device)

with torch.no_grad(), dataset.formatted_as("pt"):
    dl = DataLoader(dataset["input_ids"], batch_size=batch_size)
    for i, batch in enumerate(tqdm(dl)):
        _, cache = model.run_with_cache(batch.to(device))
        batched_neuron_activations = rearrange(cache[cache_name], "b s n -> (b s) n" )
        neuron_activations[i*batch_size*token_amount:(i+1)*batch_size*token_amount,:] = batched_neuron_activations.cpu()
        # normal dictionary
        batched_dictionary_activations = smaller_autoencoder.encode(batched_neuron_activations)
        dictionary_activations[i*batch_size*token_amount:(i+1)*batch_size*token_amount,:] = batched_dictionary_activations.cpu()


In [ ]:
from circuitsvis.activations import text_neuron_activations
# Get the activations for the best dict features
def get_feature_datapoints(feature_index, dictionary_activations, dataset, k=10, setting="max"):
    best_feature_activations = dictionary_activations[:, feature_index]
    # Sort the features by activation, get the indices
    if setting=="max":
        found_indices = torch.argsort(best_feature_activations, descending=True)[:k]
    elif setting=="uniform":
        min_value = torch.min(best_feature_activations)
        max_value = torch.max(best_feature_activations)

        # Define the number of bins
        num_bins = k

        # Calculate the bin boundaries as linear interpolation between min and max
        bin_boundaries = torch.linspace(min_value, max_value, num_bins + 1)

        # Assign each activation to its respective bin
        bins = torch.bucketize(best_feature_activations, bin_boundaries)

        # Initialize a list to store the sampled indices
        sampled_indices = []

        # Sample from each bin
        for bin_idx in torch.unique(bins):
            # Get the indices corresponding to the current bin
            bin_indices = torch.nonzero(bins == bin_idx, as_tuple=False).squeeze(dim=1)
            
            # Randomly sample from the current bin
            sampled_indices.extend(np.random.choice(bin_indices, size=1, replace=False))

        # Convert the sampled indices to a PyTorch tensor & reverse order
        found_indices = torch.tensor(sampled_indices).long().flip(dims=[0])
    else: # random
        # get nonzero indices
        nonzero_indices = torch.nonzero(best_feature_activations)[:, 0]
        # shuffle
        shuffled_indices = nonzero_indices[torch.randperm(nonzero_indices.shape[0])]
        found_indices = shuffled_indices[:k]
    datapoint_indices =[np.unravel_index(i, (datapoints, token_amount)) for i in found_indices]
    text_list = []
    full_text = []
    token_list = []
    full_token_list = []
    for md, s_ind in datapoint_indices:
        md = int(md)
        s_ind = int(s_ind)
        full_tok = torch.tensor(dataset[md]["input_ids"])
        full_text.append(model.tokenizer.decode(full_tok))
        tok = dataset[md]["input_ids"][:s_ind+1]
        text = model.tokenizer.decode(tok)
        text_list.append(text)
        token_list.append(tok)
        full_token_list.append(full_tok)
    return text_list, full_text, token_list, full_token_list

def get_neuron_activation(token, feature, model, autoencoder, setting="dictionary_basis"):
    with torch.no_grad():
        _, cache = model.run_with_cache(token.to(model.cfg.device))
        neuron_act_batch = cache[cache_name]
        if setting=="dictionary_basis":
            neuron_act_batch = rearrange(neuron_act_batch, "b s n -> (b s) n" )
            act = autoencoder.encode(neuron_act_batch)
            return act[:, feature].tolist()
        else: # neuron/residual basis
            return neuron_act_batch[0, :, feature].tolist()

def ablate_text(text, feature, model, autoencoder, setting="dictionary_basis"):
    if isinstance(text, str):
        text = [text]
    display_text_list = []
    activation_list = []
    for t in text:
        # Convert text into tokens
        if isinstance(t, str): # If the text is a list of tokens
            split_text = model.to_str_tokens(t, prepend_bos=False)
            tokens = model.to_tokens(t, prepend_bos=False)
        else: # t equals tokens
            tokens = t
            split_text = model.to_str_tokens(t, prepend_bos=False)
        seq_size = tokens.shape[1]
        if(seq_size == 1): # If the text is a single token, we can't ablate it
            continue
        original = get_neuron_activation(tokens, feature, model, autoencoder)[-1]
        changed_activations = torch.zeros(seq_size, device=device).cpu()
        for i in range(seq_size):
            # Remove the i'th token from the input
            ablated_tokens = torch.cat((tokens[:,:i], tokens[:,i+1:]), dim=1)
            changed_activations[i] += get_neuron_activation(ablated_tokens, feature, model,autoencoder, setting)[-1]
        changed_activations -= original
        display_text_list += [x.replace('\n', '\\newline') for x in split_text] + ["\n"]
        activation_list += changed_activations.tolist() + [0.0]
    activation_list = torch.tensor(activation_list).reshape(-1,1,1)
    return text_neuron_activations(tokens=display_text_list, activations=activation_list)

def visualize_text(text, feature, model, autoencoder, setting="dictionary_basis", max_activation = None):
    if isinstance(text, str):
        text = [text]
    if isinstance(feature, int):
        feature = [feature]
    display_text_list = []
    act_list = []
    for t in text:
        if isinstance(t, str): # If the text is a list of tokens
            split_text = model.to_str_tokens(t, prepend_bos=False)
            token = model.to_tokens(t, prepend_bos=False)
        else: # t are tokens
            token = t
            split_text = model.to_str_tokens(t, prepend_bos=False)
        for f in feature:
            display_text_list += [x.replace('\n', '\\newline') for x in split_text] + ["\n"]
            act_list += get_neuron_activation(token, f, model, autoencoder, setting) + [0.0]
    act_list = torch.tensor(act_list).reshape(-1,1,1)
    if(max_activation is not None):
        act_list = torch.clamp(act_list, max=max_activation)
    return text_neuron_activations(tokens=display_text_list, activations=act_list)
# Ablate the feature direction of the tokens
# token_list is a list of tokens, convert to tensor of shape (batch_size, seq_len)
from einops import rearrange
def ablate_feature_direction(tokens, feature, model, autoencoder):
    def mlp_ablation_hook(value, hook):
        # Rearrange to fit autoencoder
        int_val = rearrange(value, 'b s h -> (b s) h')

        # Run through the autoencoder
        act = autoencoder.encode(int_val)
        feature_to_ablate = feature # TODO: bring this out of the function

        # Subtract value with feature direction*act_of_feature
        dictionary_for_this_autoencoder = autoencoder.get_learned_dict()
        feature_direction = torch.outer(act[:, feature_to_ablate].squeeze(), dictionary_for_this_autoencoder[feature_to_ablate].squeeze())
        batch, seq_len, hidden_size = value.shape
        feature_direction = rearrange(feature_direction, '(b s) h -> b s h', b=batch, s=seq_len)
        value -= feature_direction
        return value

    return model.run_with_hooks(tokens, 
        fwd_hooks=[(
            cache_name, 
            mlp_ablation_hook
            )]
        )
def add_feature_direction(tokens, feature, model, autoencoder, scalar=1.0):
    def residual_add_hook(value, hook):
        feature_direction = autoencoder.decoder.weight[:, feature].squeeze()
        value += scalar*feature_direction
        return value

    return model.run_with_hooks(tokens, 
        fwd_hooks=[(
            cache_name,
            residual_add_hook
            )]
        )
def ablate_feature_direction_display(text, features, autoencoder, setting="true_tokens", verbose=False):

    if features==None:
        features = torch.tensor([best_feature])
    if isinstance(features, int):
        features = torch.tensor([features])
    if isinstance(features, list):
        features = torch.tensor(features)
    if isinstance(text, str):
        text = [text]
    text_list = []
    logit_list = []
    for t in text:
        tokens = model.to_tokens(t, prepend_bos=False)
        with torch.no_grad():
            original_logits = model(tokens).log_softmax(-1).cpu()
            ablated_logits = ablate_feature_direction(tokens, features, model, autoencoder).log_softmax(-1).cpu()
        diff_logits = ablated_logits  - original_logits# ablated > original -> negative diff
        tokens = tokens.cpu()
        if setting == "true_tokens":
            split_text = model.to_str_tokens(t, prepend_bos=False)
            gather_tokens = rearrange(tokens[:,1:], "b s -> b s 1") # TODO: verify this is correct
            # Gather the logits for the true tokens
            diff = rearrange(diff_logits[:, :-1].gather(-1,gather_tokens), "b s n -> (b s n)")
        elif setting == "max":
            # Negate the diff_logits to see which tokens have the largest effect on the neuron
            val, ind = (-1*diff_logits).max(-1)
            diff = rearrange(val[:, :-1], "b s -> (b s)")
            diff*= -1 # Negate the values gathered
            split_text = model.to_str_tokens(ind, prepend_bos=False)
            gather_tokens = rearrange(ind[:,1:], "1 s -> 1 s 1")
        split_text = split_text[1:] # Remove the first token since we're not predicting it
        if(verbose):
            text_list += [x.replace('\n', '\\newline') for x in split_text] + ["\n"]
            text_list += [x.replace('\n', '\\newline') for x in split_text] + ["\n"]
            orig = rearrange(original_logits[:, :-1].gather(-1, gather_tokens), "b s n -> (b s n)")
            ablated = rearrange(ablated_logits[:, :-1].gather(-1, gather_tokens), "b s n -> (b s n)")
            logit_list += orig.tolist() + [0.0]
            logit_list += ablated.tolist() + [0.0]
        text_list += [x.replace('\n', '\\newline') for x in split_text] + ["\n"]
        logit_list += diff.tolist() + [0.0]
    logit_list = torch.tensor(logit_list).reshape(-1,1,1)
    if verbose:
        print(f"Max & Min logit-diff: {logit_list.max().item():.2f} & {logit_list.min().item():.2f}")
    return text_neuron_activations(tokens=text_list, activations=logit_list)
def generate_text(input_text, num_tokens, model, autoencoder, feature, temperature=0.7, setting="add", scalar=1.0):
    # Convert input text to tokens
    input_ids = model.tokenizer.encode(input_text, return_tensors='pt').to(device)

    for _ in range(num_tokens):
        # Generate logits
        with torch.no_grad():
            if(setting=="add"):
                logits = add_feature_direction(input_ids, feature, model, autoencoder, scalar=scalar)
            else:
                logits = model(input_ids)

        # Apply temperature
        logits = logits / temperature

        # Sample from the distribution
        probs = torch.nn.functional.softmax(logits[:, -1, :], dim=-1)
        predicted_token = torch.multinomial(probs, num_samples=1)

        # Append predicted token to input_ids
        input_ids = torch.cat((input_ids, predicted_token), dim=-1)

    # Decode the tokens to text
    output_text = model.tokenizer.decode(input_ids[0])

    return output_text

# Logit Lens
def logit_lens(model, best_feature, smaller_dict, layer):
    with torch.no_grad():
        # There are never-used tokens, which have high norm. We want to ignore these.
        bad_ind = (model.W_U.norm(dim=0) > 20)
        feature_direction = smaller_dict[best_feature].to(device)
        # feature_direction = torch.matmul(feature_direction, model.W_out[layer]) # if MLP
        logits = torch.matmul(feature_direction, model.W_U).cpu()
    # Don't include bad indices
    logits[bad_ind] = -1000
    topk_values, topk_indices = torch.topk(logits, 20)
    top_text = model.to_str_tokens(topk_indices)
    print(f"{top_text}")
    print(topk_values)


In [ ]:
import re
import numpy as np

# Assuming you have a tokenizer like this (for example, from HuggingFace's tokenizers)
  # Replace with your tokenizer

def char_to_token_position(tokenized_output, char_idx):
    """
    Given a char-level index in the original string, find the corresponding token-level index.
    """
    token_positions = tokenized_output.char_to_token(char_idx)
    return token_positions

def find_text_in_regex(tokens, regex_pattern):
    """
    Find tokens inside parentheses and return token-level start and end indices.
    """
    detokenized_text = model.tokenizer.decode(tokens.input_ids)
    
    # Find matches in the detokenized text
    pattern = re.compile(regex_pattern)
    matches = pattern.finditer(detokenized_text)
    
    token_positions = []
    for match in matches:
        start_char, end_char = match.span()
        start_token = char_to_token_position(tokens, start_char)
        end_token = char_to_token_position(tokens, end_char)
        token_positions.append((start_token, end_token))
    return token_positions


import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

def prepare_data(best_feature_activations,  num_bins=10):
    min_value = torch.min(best_feature_activations)
    max_value = torch.max(best_feature_activations)
    bin_boundaries = torch.linspace(min_value, max_value, num_bins+1)
    bins = torch.bucketize(best_feature_activations, bin_boundaries)
    return bin_boundaries, bins

def compute_bar_positions(bin_boundaries):
    barWidth = (bin_boundaries[1] - bin_boundaries[0]) * 0.8
    r = (bin_boundaries[:-1] + bin_boundaries[1:]) / 2
    return r, barWidth

def process_data_type1(bins, dataset, num_bins, num_unique_tokens, model):
    nonzero = bins.nonzero()[:,0]
    datapoint_indices = [np.unravel_index(i, (datapoints, token_amount)) for i in nonzero]
    max_token = [dataset[int(md)]["input_ids"][int(s_ind)] for md, s_ind in datapoint_indices]
    
    ac = Counter(max_token)
    max_tokens = [token_ind for token_ind, count in ac.most_common(num_unique_tokens)]
    token_dict = {token: np.zeros(num_bins) for token in max_tokens}
    misc_count = np.zeros(num_bins)

    for i in range(1, num_bins+1):
        bin_tokens_ind = (bins == i).nonzero()[:,0]
        datapoint_indices = [np.unravel_index(ind, (datapoints, token_amount)) for ind in bin_tokens_ind]
        max_token = [dataset[int(md)]["input_ids"][int(s_ind)] for md, s_ind in datapoint_indices]
        
        total_tokens = len(max_token)
        running_count = 0
        ac = Counter(max_token)
        for token_ind, count in ac.most_common(10):
            if token_ind in max_tokens:
                token_dict[token_ind][i-1] = count
                running_count += count
        misc_count[i-1] = total_tokens - running_count

    max_text = [model.tokenizer.decode([token]) for token in max_tokens]
    return token_dict, misc_count, max_tokens, max_text

def process_data_type2(bins, dataset, num_bins, masks):
    regex_matches_vs_nonmatches = np.zeros((num_bins, 2))
    all_non_matched_tokens = []
    non_matched_all_tokens = []
    for i in range(1,num_bins+1):
        bin_tokens_ind = (bins == i).nonzero()[:,0]
        datapoint_indices = [np.unravel_index(i, (datapoints, token_amount)) for i in bin_tokens_ind]
        for md, s_ind in datapoint_indices:
            md = int(md)
            s_ind = int(s_ind)
            regex_matches_vs_nonmatches[i-1, masks[md][s_ind]] += 1
            if not masks[md][s_ind]:
                all_non_matched_tokens.append(dataset[md]["input_ids"][s_ind])
                non_matched_all_tokens.append(dataset[md]["input_ids"][:s_ind+1])
    return regex_matches_vs_nonmatches, all_non_matched_tokens

def plot_data_type1(r, barWidth, bin_boundaries, data):
    import matplotlib.font_manager as fm
    fprop = fm.FontProperties(fname='NotoSansCJKtc-Regular.otf')
    token_dict, misc_count, max_tokens, max_text = data
    colors = sns.color_palette('colorblind', len(max_tokens) + 1)
    running_sum = np.zeros(len(bin_boundaries) - 1)
    for i in range(len(max_tokens)):
        token_id_name = max_tokens[i]
        token_count_array = token_dict[token_id_name]
        text_label = max_text[i]
        plt.bar(r, token_count_array, bottom=running_sum, label=text_label, color=colors[i], width=barWidth)
        running_sum += token_count_array
    plt.bar(r, misc_count, bottom=running_sum, label='[All Other Tokens]', color=colors[-1], width=barWidth)
    plt.title(f'Token Count for Feature {feature_index} Across Activation Ranges')
    plt.xlabel('Activation')
    plt.xticks(bin_boundaries, [f"{val:.2f}" for val in bin_boundaries])
    plt.legend()
    plt.tight_layout()
    plt.show()

def plot_data_type2(r, barWidth, bin_boundaries, data, regex_pattern):
    regex_matches_vs_nonmatches, all_non_matched_tokens = data
    colors = sns.color_palette('deep', regex_matches_vs_nonmatches.shape[1] + 1)
    running_sum = np.zeros(len(bin_boundaries) - 1)
    for i in range(regex_matches_vs_nonmatches.shape[1]):
        text_label = regex_pattern if i == 0 else "non-matches"
        token_count_array = regex_matches_vs_nonmatches[:, (i+1)%2]
        plt.bar(r, token_count_array, bottom=running_sum, label=text_label, color=colors[i], width=barWidth)
        running_sum += token_count_array
    plt.title(f'Token Count for Feature {feature_index} Across Activation Ranges')
    plt.xlabel('Activation')
    plt.xticks(bin_boundaries, [f"{val:.2f}" for val in bin_boundaries])
    plt.legend()
    plt.tight_layout()
    plt.show()

import numpy as np

def generate_masks(dataset, model, token_amount, regex_pattern, extra_token_amount=2, remove_first_token=False):
    """
    Generates masks based on a regex pattern.

    Parameters:
    - dataset: dataset containing the texts to be searched
    - model: the model containing the tokenizer
    - feature_index: index of the feature (not used in the function based on given code, but kept for consistency)
    - token_amount: limit to number of tokens to be considered from the tokenized text
    - regex_pattern: the regular expression pattern to be searched for
    - extra_token_amount: additional tokens to consider

    Returns:
    - masks: a list of masks generated based on the regex_pattern
    """
    texts = dataset["text"]
    masks = []

    for text_ind, text in enumerate(texts):
        tokens = model.tokenizer(text)
        tokens['input_ids'] = tokens['input_ids'][:token_amount + extra_token_amount]
        token_positions = find_text_in_regex(tokens, regex_pattern=regex_pattern)
        mask = np.zeros(token_amount, dtype=int)  # Initial mask
        for start, end in token_positions:
            if end is None:
                print("No match found for text: ", model.tokenizer.decode(tokens.input_ids))
                print(start, end)
                continue
            if end > token_amount:
                end = token_amount
            mask[start:end] = 1
        if remove_first_token: # If we're doing the output, we never predict the first token
            mask[0] = 0
        masks.append(mask)
    return masks

In [ ]:
# Feature Search
# t = " Bratwurst, Sauerkraut, und Bier"
# t = "我们一起去玩吧"
# t = "for i in range"
# t = " I don't know about that. It is now up to Dave'"
# t = " I don't know about that. You shouldn'"
t = "x = 3\nif(x == 3):"
# t = " I like to eat them at National Institute of Justice (NIJ"
# t = " I like to eat them at National Institute of Justice ("
# t = " I like to eat them but (NIJ"
# t = "いさんさん��に"
with torch.no_grad():
    split_text = model.to_str_tokens(t, prepend_bos=False)
    token = model.to_tokens(t, prepend_bos=False)
    _, cache = model.run_with_cache(token.to(model.cfg.device))
    neuron_act_batch = cache[cache_name]
    act = smaller_autoencoder.encode(neuron_act_batch.squeeze())
    v, i = act[-1, :].topk(10)

print("Activations:",[round(val,2) for val in v.tolist()])
print("Feature_ids", i.tolist())

In [ ]:
for feature_ind in range(6):
    print(dictionary_activations[:,i[feature_ind].item()].count_nonzero())

In [ ]:
# Token level
feature_index = 1087 # dictionary 858, 101
# feature_index = 513 # PCA
# feature_index = 966 # ICA
# feature_index = 21 # neuron
num_bins=10
bin_boundaries, bins = prepare_data(dictionary_activations[:, feature_index])
# activations, bin_boundaries, bins = prepare_data(pca_dictionary_activations, feature_index)
# bin_boundaries, bins = prepare_data(just_negative[:, feature_index])
r, barWidth = compute_bar_positions(bin_boundaries)
title = f"Token Count for Feature {feature_index} Neuron Basis"
data1 = process_data_type1(bins, dataset, num_bins, num_unique_tokens=9, model=model)
# counts, _ = np.histogram(just_negative[:,feature_index].abs(), bins=bin_boundaries)

# # `counts` now holds the number of activations in each bin defined by bin_boundaries.
all_variables = (r, barWidth, bin_boundaries, data1,title)
# with open(f"images/if_feature.pkl", 'wb') as f:
#     pickle.dump(all_variables, f)
plot_data_type1(r, barWidth, bin_boundaries, data1)

In [ ]:
#Find the ablated activations when ablating this direction
ablated_dictionary_activations = torch.zeros((datapoints, token_amount))

with torch.no_grad(), dataset.formatted_as("pt"):
    dl = DataLoader(dataset["input_ids"], batch_size=batch_size)
    for i, batch in enumerate(tqdm(dl)):
        batch = batch.to(device)
        original_logits = model(batch).log_softmax(dim=-1)
        ablated_logits = ablate_feature_direction(batch, feature_index, model, smaller_autoencoder).log_softmax(dim=-1)
        logit_diff = original_logits - ablated_logits
        # Remove last token since no ground truth to compare to
        logit_diff = logit_diff[:,:-1,:]
        # Remove first token since not predicting it
        tokens_to_search_for = batch[:,1:]
        # Find the logit difference for the token we are looking for
        correct_logit_diff = logit_diff.gather(dim=-1, index=tokens_to_search_for.unsqueeze(-1)).squeeze(-1)
        # Add to dictionary
        ablated_dictionary_activations[i*batch_size:(i+1)*batch_size,1:] = correct_logit_diff.cpu()

In [ ]:
# import torch

# # Assuming vocab_size, model, tokens, and hook are already defined
# causal_tokens = np.zeros(model.cfg.d_vocab)
# k = 10
# with torch.no_grad(), dataset.formatted_as("pt"):
#     dl = DataLoader(dataset["input_ids"], batch_size=batch_size)
#     for i, batch in enumerate(tqdm(dl)):
#         batch = batch.to(device)
#         original_logits = model(batch).log_softmax(dim=-1)
#         ablated_logits = ablate_feature_direction(batch, feature_index, model, smaller_autoencoder).log_softmax(dim=-1)

#         # Find tokens that highly activate in original, and see their corresponding difference in ablated for just those k tokens
#         topk_logits_original_indices = torch.topk(original_logits, k, dim=1).indices
#         original_values_at_topk_ablated = torch.gather(ablated_logits, 1, topk_logits_original_indices)
#         diff_original_ablated = original_logits.gather(1, topk_logits_original_indices) - original_values_at_topk_ablated

#         # Add to causal tokens
#         causal_tokens[topk_logits_original_indices.cpu().numpy()] += diff_original_ablated.squeeze().cpu().numpy()

#         # Repeat for ablated model
#         topk_logits_ablated_indices = torch.topk(ablated_logits, k, dim=1).indices
#         ablated_values_at_topk_original = torch.gather(original_logits, 1, topk_logits_ablated_indices)
#         diff_ablated_original = ablated_logits.gather(1, topk_logits_ablated_indices) - ablated_values_at_topk_original

#         # Add to causal tokens (Note: this operation will modify the original entries, hence the need to recompute them)
#         causal_tokens[topk_logits_ablated_indices.cpu().numpy()] += diff_ablated_original.squeeze().cpu().numpy()

#         # Find tokens with the most positive and negative difference
#         worse_tokens_ind = np.argpartition(causal_tokens, -k)[-k:]
#         best_tokens_ind = np.argpartition(causal_tokens, k)[:k]

#         worse_tokens_val = causal_tokens[worse_tokens_ind]
#         best_tokens_val = causal_tokens[best_tokens_ind]

#         # Convert token indices to text (assuming a decode function exists)
#         worse_tokens_text = [model.tokenizer.decode([ind]) for ind in worse_tokens_ind]
#         best_tokens_text = [model.tokenizer.decode([ind]) for ind in best_tokens_ind]
#         break

In [ ]:
ablated_dictionary_activations.count_nonzero(), (ablated_dictionary_activations < 0).count_nonzero()

In [ ]:
ablated_dictionary_activations.shape, correct_logit_diff.shape

In [ ]:
# make ablated_dictionary_activations go from shape (a,b) -> (a*b)
rearrange(ablated_dictionary_activations, 'datapoints token_amount -> (datapoints token_amount)')

In [ ]:
rearranged_ablated = rearrange(ablated_dictionary_activations, 'datapoints token_amount -> (datapoints token_amount)')
total_below_zero = (rearranged_ablated < 0).count_nonzero().item()
total_above_zero = (rearranged_ablated > 0).count_nonzero().item()
print("Total number of negative values: ", total_below_zero)
print("Total number of positive values: ", total_above_zero)
positive_threshold = 0.1
negative_threshold = 0.1
positive_only = rearranged_ablated.clone().clamp(min=0)
positive_only[positive_only < positive_threshold] = 0
negative_only = rearranged_ablated.clone().clamp(max=0).abs()
negative_only[negative_only < negative_threshold] = 0
positive_only.count_nonzero(), negative_only.count_nonzero()

In [ ]:
positive_only.count_nonzero() / total_above_zero

In [ ]:
# Token level
num_bins=10
# bin_boundaries, bins = prepare_data(negative_only)
bin_boundaries, bins = prepare_data(positive_only)
r, barWidth = compute_bar_positions(bin_boundaries)
title = f"Token Count for Feature {feature_index} Neuron Basis"
data1 = process_data_type1(bins, dataset, num_bins, num_unique_tokens=9, model=model)
all_variables = (r, barWidth, bin_boundaries, data1, title, total_above_zero, positive_threshold)
# with open(f"images/if_feature_effect.pkl", 'wb') as f:
#     pickle.dump(all_variables, f)
plot_data_type1(r, barWidth, bin_boundaries, data1)

In [ ]:
# Token level
feature_index = 556 # dictionary
# feature_index = 513 # PCA
# feature_index = 966 # ICA
# feature_index = 21 # neuron
num_bins=10
bin_boundaries, bins = prepare_data(negative_only)
# bin_boundaries, bins = prepare_data(positive_only)
# activations, bin_boundaries, bins = prepare_data(pca_dictionary_activations, feature_index)
# bin_boundaries, bins = prepare_data(just_negative[:, feature_index])
r, barWidth = compute_bar_positions(bin_boundaries)
title = f"Token Count for Feature {feature_index} Neuron Basis"
data1 = process_data_type1(bins, dataset, num_bins, num_unique_tokens=9, model=model)
# counts, _ = np.histogram(just_negative[:,feature_index].abs(), bins=bin_boundaries)

# # `counts` now holds the number of activations in each bin defined by bin_boundaries.
# all_variables = (r, barWidth, bin_boundaries, data1, title, total_below_zero, negative_threshold)
# with open(f"images/apostrophe_effect_negative.pkl", 'wb') as f:
#     pickle.dump(all_variables, f)
plot_data_type1(r, barWidth, bin_boundaries, data1, model)

In [ ]:
# feature_index = 2808
# regex_pattern =  r'(?<=\().*?(?=\))'
# regex_pattern = r'\((?=[A-Z]{1,})'
regex_pattern = r'\((?=[A-Z]{1,})'
masks = generate_masks(dataset, model, token_amount, regex_pattern)

activations, bin_boundaries, bins = prepare_data(rearranged_ablated)
bin_boundaries, bins = prepare_data()
r, barWidth = compute_bar_positions(bin_boundaries)
data2 = process_data_type2(bins, dataset, num_bins, masks)
plot_data_type2(r, barWidth, bin_boundaries, data2, regex_pattern)

In [ ]:
regex_nonzero, feature_nonzero = np.array(masks).flatten().nonzero()[0], dictionary_activations[:, feature_index].numpy().nonzero()[0]
index_matches = np.intersect1d(regex_nonzero, feature_nonzero)
false_positives = np.setdiff1d(regex_nonzero, index_matches)
false_negatives = np.setdiff1d(feature_nonzero, index_matches)
print(index_matches.shape, false_positives.shape, false_negatives.shape)
print(f"Matches: {index_matches.shape[0]}, False Positives: {false_positives.shape[0]}, False Negatives: {false_negatives.shape[0]}")
found_indices = false_positives[:20]

datapoint_indices =[np.unravel_index(i, (datapoints, token_amount)) for i in found_indices]
text_list = []
full_text = []
token_list = []
full_token_list = []
for md, s_ind in datapoint_indices:
    md = int(md)
    s_ind = int(s_ind)
    full_tok = torch.tensor(dataset[md]["input_ids"])
    full_text.append(model.tokenizer.decode(full_tok))
    tok = dataset[md]["input_ids"][:s_ind+1]
    text = model.tokenizer.decode(tok)
    text_list.append(text)
    token_list.append(tok)
    full_token_list.append(full_tok)
# text_list, full_text, token_list, full_token_list
visualize_text(text_list, feature_index, model)

In [ ]:
N = 118
best_feature = int(max_indices[N])
# best_feature = 21 # Change this one for global index (N is sorted by MCS)
# best_feature = 1630 # Feature_ids [1359, 1764, 475, 1815, 513, 1184, 1100, 995, 1001, 365]


feature_setting = "dictionary"
# feature_setting = "pca"
# feature_setting = "ica"
# feature_setting = "neuron"
setting="dictionary_basis"
if(feature_setting == "dictionary"):
    autoencoder = smaller_autoencoder
    activations = dictionary_activations
elif(feature_setting == "pca"):
    autoencoder = pca_topk
    activations = pca_dictionary_activations
elif(feature_setting == "ica"):
    autoencoder = ica_topk
    activations = ica_dictionary_activations
elif(feature_setting == "neuron"):
    autoencoder = smaller_autoencoder # Doesn't matter actually
    activations = neuron_activations
    setting = "neuron_basis"

# autoencoder = pca_topk
print(f"Feature index: {best_feature}")
print(f"MCS: {max_cosine_similarities[best_feature]}")
text_list, full_text, token_list, full_token_list = get_feature_datapoints(best_feature, activations, dataset, setting="uniform")
# text_list, full_text, token_list, full_token_list = get_feature_datapoints(best_feature, activations, dataset, setting="max")
visualize_text(text_list, best_feature, model, autoencoder, setting=setting)
# visualize_text(full_text, best_feature, model, autoencoder)

In [ ]:
ablate_text(text_list, best_feature, model, autoencoder)

In [ ]:
ablate_feature_direction_display(full_text, best_feature, autoencoder)

In [ ]:
logit_lens(model,best_feature, smaller_dict, layer)